# 4.2 CNN을 사용한 이미지 분류

CNN은 합성곱으로 부터 ReLU 등의 활성화 함수를 적용하는 과정을 여러 번 실시하면 된다.
컨벌루션 처리 후에 위치 감도를 높이는 pooling을 사용하거나 Dropout Batch Normalization을 사용하는 경우도 있다.


## 4.2.1 Fashion-MNIST

In [28]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm
# tqdm은 진행상황을 바로 보여주는 것

from torchvision.datasets import FashionMNIST
from torchvision import transforms

# 훈련용 데이터 가져오기 초기 상태에선 Python Imagin Library 이미지 형식으로 Dataset을 만들어버린다.
# 따라서 transforms.ToTensor를 사용해 텐서로 변환
fashion_mnist_train = FashionMNIST("/data/FashionMNIST", train=True, download=True, transform=transforms.ToTensor())
# 검증용 데이터 가져오기
fashion_mnist_test = FashionMNIST("/data/FashionMNIST", train=False, download=True, transform=transforms.ToTensor())

# 배치 크기가 128인 DataLoader를 각각 작성
batch_size = 128
train_loader = DataLoader(fashion_mnist_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(fashion_mnist_train, batch_size = batch_size, shuffle = False)

In [29]:
# (N, C, H, W)형식의 텐서를 (N, C*H*W)로 늘리는 계층
# 합성곱 출력을 MLP에 전달할 때 필요
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)
    
# 5x5의 커널을 사용해서 처음에 32개 그다음에 64개 채널 작성
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

# 합성곱에 의해 최종적으로 이미지 크기가 어떤지를 더미 데이터에 넣어서 확인한다.
test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

# 2층 MLP
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)

# 최종 CNN
net = nn.Sequential(
    conv_net,
    mlp
)


In [30]:
# 평가용 헬퍼 함수
def eval_net(net, data_loader, device = "cpu"):
    # Dropout및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x,y in data_loader:
        # to 메소드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 분류를 예측
        # 여기선 forward(추론) 계산이 전부이므로 자동 미분에 필요한 처리는 off로 설정해서 불필요한 계산을 제한.
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
        
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

#훈련용 헬퍼 함수
def train_net(net, train_loader, test_loader, optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss(), n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행바를 표시
        for i, (xx,yy) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)
        
        # 검증 데이터의 예측 정확도
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

# 신경망의 모든 파라미터 GPU로 전송
net.to("cuda:0")

#훈련 실행
train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0")

100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:11<00:00, 40.46it/s]


0 0.47020614300018704 0.83595 0.8963500261306763


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:11<00:00, 42.49it/s]


1 0.3164025517100962 0.8854333333333333 0.9110000133514404


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:11<00:00, 42.48it/s]


...

17 0.1608779950815643 0.9386 0.9655333161354065


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:11<00:00, 41.79it/s]


18 0.15791023816340244 0.9403833333333333 0.9678333401679993


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:11<00:00, 41.74it/s]


19 0.15281154769353378 0.9424833333333333 0.9677166938781738


# 4.3 전이학습(Transfer Learning)


In [31]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm
from torchvision.datasets import ImageFolder
from torchvision import transforms

# ImageFolder 함수를 사용해서 Dataset 작성
train_imgs = ImageFolder("PyTorch_data/taco_and_burrito/train/", transform=transforms.Compose([transforms.RandomCrop(224), 
                                                                                               transforms.ToTensor()]))
test_imgs = ImageFolder("PyTorch_data/taco_and_burrito/test/", transform=transforms.Compose([transforms.RandomCrop(224), 
                                                                                               transforms.ToTensor()]))
# DataLoader 작성
train_loader = DataLoader(train_imgs, batch_size=32, shuffle = True)
test_loader = DataLoader(test_imgs, batch_size=32, shuffle = True)

In [32]:
print('1 -> ', train_imgs.classes)

1 ->  ['burrito', 'taco']


In [33]:
print('2 -> ', train_imgs.class_to_idx)

2 ->  {'burrito': 0, 'taco': 1}


In [34]:
from torchvision import models

# 사전 학습이 완료된 resnet18 불러오기
net = models.resnet18(pretrained = True)
# 모든 파라미터를 미분 대상에서 제외한다
for p in net.parameters():
    p.requres_grad = False

# 마지막 선형 계층을 변경한다.
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

In [35]:
def eval_net(net, data_loader, device="cpu"):
    # Dropout 및 BatchNorm 무효화
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # to 메소드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 분류를 예측
        # 여기선 forward(추론) 계산이 전부이므로 자동 미분에 필요한 처리는 off로 설정해서 불필요한 계산을 제한다.
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys==ypreds).float().sum() / len(ys)
    return acc.item()

def train_net(net, train_loader, test_loader, only_fc = True, optimizer_cls=optim.Adam, 
              loss_fn=nn.CrossEntropyLoss(), n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행 바를 표시
        for i, (xx,yy) in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h,yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)
        
        # 검증 데이터의 예측 정확도
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush = True)
        
net.to("cuda:0")

# 훈련실행
train_net(net, train_loader, test_loader, n_iter=20, device = "cuda:0")

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:07<00:00,  3.22it/s]


0 0.6602973436767404 0.6348314606741573 0.7666667103767395


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  3.86it/s]


1 0.5001959380778399 0.7879213483146067 0.8000000715255737


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  3.86it/s]


...

17 0.3201365105130456 0.8609550561797753 0.8166667222976685


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.73it/s]


18 0.29887542805888434 0.8890449438202247 0.8166667222976685


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.76it/s]


19 0.3002345995469527 0.8764044943820225 0.8833333849906921


# 4.4 CNN 회귀 모델을 사용한 이미지 해상도 향상
얼굴 이미지의 해상도를 향상시켜 본다.
확대한 이미지를 bilinear와 CNN망을 통해 거친 이미지를 확인해본다

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder

class DownSizePairImageFolder(ImageFolder):
    def __init__(self, root, transform=None, large_size=128, small_size=32, **kwds):
        super().__init__(root, transform=transform, **kwds)
        self.large_resizer = transforms.Resize(large_size)
        self.small_resizer = transforms.Resize(small_size)
        
    def __getitem__(self, index):
        path, _ = self.imgs[index]
        img = self.loader(path)
        
        # 읽은 이미지를 128x128픽셀과 32x32픽셀로 리사이즈
        large_img = self.large_resizer(img)
        small_img = self.small_resizer(img)
        
        # 기타 변환 적용
        if self.transform is not None:
            large_img = self.transform(large_img)
            small_img = self.transform(small_img)
            
        # 32픽셀의 이미지와 128 픽셀의 이미지 반환
        return small_img, large_img

In [10]:
train_data = DownSizePairImageFolder("PyTorch_data/lfw-deepfunneled/train", transform=transforms.ToTensor())
test_data = DownSizePairImageFolder("PyTorch_data/lfw-deepfunneled/test", transform=transforms.ToTensor())

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

In [11]:
net = nn.Sequential(
    nn.Conv2d(3, 256, 4, stride = 2, padding = 1),
    nn. ReLU(),
    nn.BatchNorm2d(256),
    nn.Conv2d(256, 512, 4, stride = 2, padding = 1),
    
    nn. ReLU(),
    nn.BatchNorm2d(512),
    nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
    nn. ReLU(),
    nn.BatchNorm2d(256),
    nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
    nn. ReLU(),
    nn.BatchNorm2d(128),
    nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
    nn. ReLU(),
    nn.BatchNorm2d(64),
    nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
    
)

In [12]:
# PSNR 계산
import math
def psnr(mse, max_v = 1.0):
    return 10 * math.log10(max_v**2 / mse)

# 평가 헬퍼 함수
def eval_net(net, data_loader, device = "cpu"):
    # Dropout 및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x,y in data_loader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            y_pred = net(x)
        ys.append(y)
        ypreds.append(y_pred)
    # 미니 배치 단위로 예측 결과 등을 하나로 모은다
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    
    # 예측 정확도(MSE) 계산
    score = nn.functional.mse_loss(ypreds, ys).item()
    return score

# 훈련 헬퍼 함수
def train_net(net, train_loader, test_loader, optimizer_cls = optim.Adam, loss_fn=nn.MSELoss(), n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        score = 0
        # 시간이 많이 걸리므로 tqdm을 이용해서 진행 바 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            y_pred = net(xx)
            loss = loss_fn(y_pred, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
        train_losses.append(running_loss / len(train_loader))
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], psnr(train_losses[-1]), psnr(val_acc[-1]), flush=True)


In [13]:
net.to("cuda:0")
train_net(net, train_loader, test_loader, device="cuda:0")

100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [01:00<00:00,  6.79it/s]


0 0.025676979207431484 15.904560705841465 22.92659900570736


100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [00:58<00:00,  6.96it/s]


1 0.00387565112982428 24.116553232650475 25.086853972260915


100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [00:59<00:00,  6.92it/s]


...

7 0.002425537399963583 26.15192024491811 26.993737295052476


100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [00:59<00:00,  6.87it/s]


8 0.002351158089346018 26.287181683985192 26.81978765596081


100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [00:59<00:00,  6.83it/s]


9 0.002296410654705646 26.389504467943446 26.871352218514303


In [21]:
from torchvision.utils import save_image

# 테스트 데이터로부터 랜덤으로 4개씩 추출하는 DataLoader
random_test_loader = DataLoader(test_data, batch_size=4, shuffle=True)
# DataLoader를 파이썬의 이터레이터로 변환해서 4개의 예로 추출
it = iter(random_test_loader)
x, y = next(it)

# Bilinear를 사용한 확대
bl_recon = torch.nn.functional.upsample(x, 128, mode="bilinear", align_corners=True)
# CNN으로 확대
yp = net(x.to("cuda:0")).to("cpu")

# torch.cat로 원본, Bilinear, CNN이미지를 결합하고 save_image로 결합한 이미지를 출력
save_image(torch.cat([y, bl_recon, yp], 0), "cnn_upscale.jpg", nrow = 4)

여기서의 nn.Sequential()은 Conv2d 두개, ConTransposed2d를 네개 연결한 CNN망이고 이것으로 이미지가 4배 확대된다.
활성화 함수에 ReLU를 사용하고 Batch Normalization을 사용한다.
원래 이미지와 확대한 이미지 사이의 MSE를 최소화 하기 위해 CNN을 훈련한다.
음성이나 이미지에서는 MSE대신 PSNR이라는 말을 자주 사용한다.